In [2]:
import os
import json
import random
import numpy as np
import pandas as pd
import argparse
import collections
from PIL import Image
from glob import glob
import sklearn
import sklearn.metrics

In [3]:
import torch
import torch.nn as nn
from torch.nn import CTCLoss
from torch.utils import data
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms

In [4]:
devise = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ["CUDA_VISIBLE_DEVICES"]="3"

DATASET_PATH = '/workspace/01_data/01_handwritten/02_processed'

In [5]:
class CustomDataset(data.Dataset):
    def __init__(self, root, phase='train', transform=None, target_transform=None):
        # 경로 생성 후 생성된 경로의 이미지 파일을 불러와 정렬한 다음 저장
        self.root = os.path.join(root, 'd2', phase)
        self.labels = []
        self.transform = transform
        self.target_transform = target_transform
        annotations = None
        
        if phase == 'val' :
            self.root = os.path.join(root, 'd2', 'train')
        
        # 라벨 데이터인 json 파일을 불러와 저장한 다음 json 파일 안의 딕셔너리를 파일 이름 순으로 정렬
        with open(os.path.join(self.root, 'labels.json'), 'r') as label_json :
            label_json = json.load(label_json)
            annotations = label_json['annotations']
        annotations = sorted(annotations, key=lambda x: x['file_name'])
        
        self.imgs = sorted(glob(self.root + '/images' + '/*.png'))
        
        if phase == 'train':
            annotations = annotations[:int(0.9*len(annotations))]
            self.imgs = self.imgs[:int(0.9*len(self.imgs))]
        elif phase == 'val':
            annotations = annotations[int(0.9*len(annotations)):]
            self.imgs = self.imgs[int(0.9*len(self.imgs)):]
            
        for annon in annotations:
            if phase == 'test':
                self.labels.append('dummy')
            else:
                self.labels.append(anno['text'])
    
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, index) :
        assert index <= len(self), 'index range error'
        img_path = self.imgs[index]
        #이미지 모드 변경 흰배경에 검은 글씨 뿐이므로 그레이 스케일 ('L') 지정
        img = Image.open(img_path).convert('L')
        
        label = self.labels[index]
        
        if self.transform is not None:
            img = self.transform(img)
            
        if self.target_transform is not None:
            label = self.target_transform(label)
            
        return (img, label)
    
    # CustomDataset 클래스의 __init__ 매서드에서 정의한 self.root 출력
    def get_root(self):
        return self.root
    
    # 해당 index의 이미지 파일의 경로 출력
    def get_img_path(self, index):
        return self.imgs[index]
    
    
# 이미지 사이즈 변경 (resize), 이중선형보간(bilinear interpolation), 텐서 변환, 표준화(normalize)
    
    

In [ ]:
#